In [ ]:
import gc
import re
import os
import copy
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from skimage.io import imread
from sklearn.neighbors import KNeighborsClassifier as ks

In [ ]:
arr=[]
labels = []
i=0
j=0
drive.mount('/content/drive')
for root,di,files in os.walk("/content/drive/MyDrive/Colab Notebooks/dataset"): 
  for name in files:
    if name.endswith("pgm"):
      root_folder = os.path.split(root)
      res=re.findall('(\d+)',root_folder[1])
      image = imread(os.path.join(root,name))
      image=image.flatten(order='C')
      arr.append(image)
      labels.append(int(res[0]))
      i=i+1
arr=np.array(arr)   
labels=np.array(labels)    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(labels.shape)       
print(arr.shape)
print(labels[300])

(400,)
(400, 10304)
1


In [ ]:
# adding the data to a dataframe
df = pd.DataFrame(arr) 
df['label'] = pd.DataFrame(labels)
# sorting the data according to labels
df.sort_values("label", inplace=True)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,10295,10296,10297,10298,10299,10300,10301,10302,10303,label
306,48,49,45,47,49,57,39,42,53,49,...,44,40,41,49,42,44,47,46,46,1
308,42,41,44,46,48,39,37,37,33,37,...,33,39,39,40,41,43,42,43,41,1
309,44,43,32,32,30,30,38,40,48,66,...,42,42,40,33,29,37,43,43,37,1
300,64,76,80,53,34,72,60,66,66,50,...,28,34,32,35,34,35,35,37,39,1
301,34,34,33,32,38,40,39,49,54,57,...,44,38,30,37,30,36,37,40,33,1


In [ ]:
# seperating images from labels
Y = np.array(df['label'])
df = df.drop(axis=1, labels="label")

In [ ]:
# creating the data matrix D
D = [[] for i in range(40)]
for i in range(0, 400): D[i//10].append(df.iloc[i])
D = np.array(D)
print(D.shape)

(40, 10, 10304)


In [ ]:
X_train = []
X_test =  []  
labels_train = []
labels_test =  []  

for i in range(0,400):
    if((i%2==0)):
       X_test.append(arr[i,:])
       labels_test.append(labels[i])
    else:
       X_train.append(arr[i,:])
       labels_train.append(labels[i])
 
print(X_test[0])
print(labels_test[0])

[112 115 120 ...  53  53  54]
4


In [ ]:
def LDAOLD(D, Y=None, noOfDims=None):
    # initializing the variables
    summ, lenn = 0, 0
    # computing the mean for each class
    mu = []
    for x in D: 
        mu.append(np.mean(x, axis=0, keepdims=True).T)
        # calculating the sum and overall number
        # elements for overall mu
        summ += sum(x)
        lenn += len(x)
    mu = np.array(mu)
    overallMu = summ / lenn
    # computing the between class scatter matrices
    SB = 0
    for i in range(len(mu)): SB += len(D[i]) * np.matmul((mu[i]-overallMu), (mu[i]-overallMu).T)
    # centering the class matrices
    Z = np.array([D[i] - mu[i].T for i in range(len(D))])
    # computing the within class scatter matrices
    S = np.array([np.matmul(Z[i].T, Z[i]) for i in range(len(Z))])
    # performing eigenvalue decomposition
    eigValues, eigVectors = np.linalg.eigh(np.matmul(np.linalg.inv(sum(S)), SB))
    # returning the dominant eigen vectors
    return eigVectors[::-1][:noOfDims].T

In [ ]:
def LDA(D, Y=None, noOfDims=None):
    # initializing the variables
    summ, lenn = 0, 0
    # computing the mean for each class
    mu = []
    for x in D: 
        mu.append(np.mean(x, axis=0, keepdims=True).T)
        # calculating the sum and overall number
        # elements for overall mu
        summ += np.sum(x, axis=0)
        lenn += len(x)
    mu = np.array(mu)
    overallMu = np.array(summ / lenn).reshape(mu[0].shape)
    # computing the between class scatter matrices
    SB = 0
    for i in range(len(mu)): SB += len(D[i]) * np.matmul((mu[i]-overallMu), (mu[i]-overallMu).T)
    # centering the class matrices
    Z = np.array([D[i] - mu[i].T for i in range(len(D))])
    # computing the within class scatter matrices
    S = 0
    for i in range(len(Z)): S += np.matmul(Z[i].T, Z[i])
    # performing eigenvalue decomposition
    eigValues, eigVectors = np.linalg.eigh(np.matmul(np.linalg.inv(S), SB))
    # returning the dominant eigen vectors
    return eigVectors[::-1][:noOfDims].T

In [ ]:
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES
W = LDA(D, noOfDims=39)
# DO NOT RUN THIS CELL IF YOU ALREADY HAVE THE PICKLE FILES

In [ ]:
# saving the LDA output for later use
file_name = 'LDA_output.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(W, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "LDA_output.pkl"


In [ ]:
file_name = 'LDA_output.pkl'
with open(file_name, 'rb') as file:
    W  = pickle.load(file)
    print(f'Object successfully loaded from "{file_name}"')

Object successfully loaded from "LDA_output.pkl"


In [ ]:
print(eigVals)

[-4.68059819e+18 -3.45273497e+18 -2.22037813e+18 ...  2.22071896e+18
  3.46377157e+18  4.65618892e+18]


In [ ]:
W - eigVecs[::-1][:39].T

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
def projectData(X_train, X_test, W):
    X_train = np.array([np.matmul(W.T, X_train[i]) for i in range(X_train.shape[0])])
    X_test = np.array([np.matmul(W.T, X_test[i]) for i in range(X_test.shape[0])])
    return X_train, X_test

In [ ]:
reg = ks().fit(X_train, labels_train)
train_score = reg.score(X_train, labels_train)
test_score = reg.score(X_test, labels_test)
print("Before LDA")
print("Training Set Accuracy: {}\nTesting  Set Accuracy: {}\n\n\n".format(train_score, test_score))

print("After LDA")
X_train_red, X_test_red = projectData(np.array(X_train), np.array(X_test), W)
reg = ks().fit(X_train_red, labels_train)
train_score = reg.score(X_train_red, labels_train)
test_score = reg.score(X_test_red, labels_test)
print("Training Set Accuracy: {}\nTesting  Set Accuracy: {}".format(train_score, test_score))

Before LDA
Training Set Accuracy: 0.875
Testing  Set Accuracy: 0.81



After LDA
Training Set Accuracy: 0.825
Testing  Set Accuracy: 0.76


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, labels_train)
knn.score(X_test, labels_test)
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train_red, labels_train)
knn.score(X_test_red, labels_test)
knn.score(X_test_red, labels_test)

0.88

In [ ]:
def KMC(X, k, epsilon=0.01, maxIter=100):
  # initializing the error to be max at first
  err = float('inf')
  # randomly initializing the centroids
  centroid = [X[np.random.randint(0, len(X)-1)] for i in range(k)]
  while maxIter > 0 and err >= epsilon:
    maxIter -= 1
    cluster = [[] for i in range(k)]
    # copying the old centroids
    oldCentroid = copy.deepcopy(centroid)
    # assigning each data point to closest cluster
    for x in X:
      minn, j = float('inf'), 0
      for i in range(len(centroid)):
        dist = np.linalg.norm(x-centroid[i]) ** 2
        if dist < minn: minn, j = dist, i
      cluster[j].append(x)
    # updating the centroids
    for i in range(len(cluster)):
      if len(cluster[i]) != 0: centroid[i] = sum(cluster[i]) / len(cluster[i])
    # if centroids didn't change, then terminate
    err = 0
    for i in range(len(centroid)):
      err += np.linalg.norm(centroid[i] - oldCentroid[i]) ** 2
  return cluster